In [92]:
import pandas as pd
import numpy as np
from WrongData import introduce_filthy_data
from sklearn.impute import KNNImputer

## Adding filthy data

In [93]:
df = pd.read_csv("./dataset/spotify_songs_restructured.csv", sep=",")
new_data = introduce_filthy_data(df, 1500)
new_data.to_csv("./dataset/spotify_songs_restructured_filthy.csv", index=False)
new_data.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,31198.000000,31188.000000,31155.000000,31185.000000,31249.000000,31181.000000,31218.000000,31239.000000,31069.000000,31132.000000,31222.000000,31194.000000,31283.000000
mean,42.424386,0.654613,0.698889,5.378355,-6.717303,0.566082,0.106882,0.175109,0.085390,0.190458,0.510616,120.823562,225924.106991
std,24.983660,0.144961,0.180736,3.615573,2.990126,0.495622,0.101020,0.219549,0.225106,0.154424,0.233293,26.886354,59884.784021
min,0.000000,0.000000,0.000175,0.000000,-46.448000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4000.000000
25%,24.000000,0.563000,0.582000,2.000000,-8.170000,0.000000,0.041000,0.015100,0.000000,0.092800,0.331000,99.947750,187840.000000
50%,45.000000,0.671000,0.721000,6.000000,-6.164000,1.000000,0.062500,0.080000,0.000016,0.127000,0.512000,121.974000,216120.000000
75%,62.000000,0.760000,0.840500,9.000000,-4.644000,1.000000,0.131000,0.254000,0.004900,0.249000,0.694000,133.756250,253731.000000
max,100.000000,0.983000,1.000000,11.000000,1.275000,1.000000,0.918000,0.994000,0.994000,0.996000,0.991000,239.440000,517810.000000


## Transforming the data

In [94]:
# nan van naam weghalen
df_clean = new_data.dropna(subset=['track_name', 'track_artist', 'track_album_name', 'track_album_release_date', 'playlist_name',
                                   'playlist_genre', 'playlist_subgenre'])
# df_clean = new_data.dropna(subset=['track_album_release_date'])
df_clean.head(10)

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
1,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,dance pop,0.726,0.815,11.0,-4.969,1.0,0.0373,0.0724,0.004210,0.357,0.693,NaN,162600.0
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718,0.930,7.0,-3.778,NaN,0.1020,0.0287,0.000009,0.204,0.277,121.956,169093.0
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.0,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650,0.833,1.0,-4.672,1.0,0.0359,0.0803,0.000000,NaN,0.725,123.976,189052.0
6,Never Really Over - R3HAB Remix,Katy Perry,62.0,Never Really Over (R3HAB Remix),2019-07-26,Pop Remix,pop,dance pop,0.449,0.856,5.0,-4.788,0.0,NaN,0.1870,0.000000,0.176,0.152,112.648,187675.0
7,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.0,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29,Pop Remix,pop,dance pop,0.542,0.903,4.0,-2.419,0.0,0.0434,0.0335,0.000005,NaN,0.367,127.936,207619.0
9,If I Can't Have You - Gryffin Remix,Shawn Mendes,67.0,If I Can't Have You (Gryffin Remix),2019-06-20,Pop Remix,pop,dance pop,0.642,0.818,2.0,-4.552,1.0,0.0320,0.0567,0.000000,NaN,0.590,124.957,253040.0
10,Cross Me (feat. Chance the Rapper & PnB Rock) ...,Ed Sheeran,58.0,Cross Me (feat. Chance the Rapper & PnB Rock) ...,2019-06-21,Pop Remix,pop,dance pop,0.679,0.923,6.0,-6.500,1.0,0.1810,0.1460,NaN,0.124,0.752,121.984,207894.0
11,Hate Me - R3HAB Remix,Ellie Goulding,67.0,Hate Me (R3HAB Remix),2019-08-16,Pop Remix,pop,dance pop,0.437,NaN,8.0,-4.918,1.0,0.0554,0.1480,0.000000,0.133,0.329,123.125,203733.0
12,Body On My,Loud Luxury,67.0,Body On My,2019-03-29,Pop Remix,pop,dance pop,0.744,0.726,1.0,-4.675,1.0,0.0463,0.0399,NaN,0.374,0.687,121.985,192507.0
15,South of the Border (feat. Camila Cabello & Ca...,Ed Sheeran,NaN,South of the Border (feat. Camila Cabello & Ca...,2019-12-11,Pop Remix,pop,dance pop,0.805,0.835,0.0,-4.603,1.0,0.0896,0.1300,0.000005,0.365,0.722,125.028,188230.0


In [95]:
new_data.isnull().sum()

track_name                  1668
track_artist                1559
track_popularity            1635
track_album_name            1637
track_album_release_date    1696
playlist_name               1635
playlist_genre              1644
playlist_subgenre           1693
danceability                1645
energy                      1678
key                         1648
loudness                    1584
mode                        1652
speechiness                 1615
acousticness                1594
instrumentalness            1764
liveness                    1701
valence                     1611
tempo                       1639
duration_ms                 1550
dtype: int64

### Imputing missing values

In [96]:
df_clean = df_clean.fillna({
    "danceability": df_clean["danceability"].mean(), 
    "energy": df_clean.energy.mean(), 
    "key": df_clean.key.mean(), 
    "loudness": df_clean.loudness.mean(), 
    "mode": df_clean['mode'].mean(), 
    "speechiness": df_clean.speechiness.mean(), 
    "acousticness": df_clean.acousticness.mean(), 
    "instrumentalness": df_clean.instrumentalness.mean(), 
    "liveness": df_clean.liveness.mean(), 
    "valence": df_clean.valence.mean(), 
    "tempo": df_clean.tempo.mean(), 
    "duration_ms": df_clean.duration_ms.mean(),
    "track_popularity": df_clean.track_popularity.mean()
})

df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,21313.000000,21313.000000,21313.000000,21313.000000,21313.000000,21313.000000,21313.000000,21313.000000,21313.000000,21313.000000,21313.000000,21313.000000,21313.000000
mean,42.685807,0.654245,0.699373,5.377047,-6.713885,0.566451,0.106450,0.174117,0.085092,0.191022,0.511568,120.757315,225796.836563
std,23.973837,0.139443,0.172336,3.475577,2.872429,0.475982,0.096429,0.210348,0.214685,0.149645,0.224828,25.786040,57638.018063
min,0.000000,0.077100,0.000175,0.000000,-46.448000,0.000000,0.022400,0.000001,0.000000,0.009360,0.000010,35.477000,31875.000000
25%,27.000000,0.573000,0.596000,2.000000,-7.952000,0.000000,0.042000,0.017700,0.000000,0.095300,0.346000,100.066000,190185.000000
50%,43.000000,0.655000,0.701000,5.377047,-6.418000,1.000000,0.068100,0.097000,0.000042,0.138000,0.511568,120.757315,220590.000000
75%,61.000000,0.752000,0.831000,8.000000,-4.770000,1.000000,0.121000,0.230000,0.032100,0.230000,0.679000,130.121000,249440.000000
max,100.000000,0.983000,1.000000,11.000000,0.326000,1.000000,0.918000,0.994000,0.994000,0.996000,0.991000,239.440000,517125.000000


### outliers removal

In [97]:
# Define the columns where you want to remove outliers
columns_to_clean = ["danceability", "energy", "key", "loudness", "mode", 
                    "speechiness", "acousticness", "instrumentalness", 
                    "liveness", "valence", "tempo", "duration_ms"]

# Loop through each column and apply the custom quantile-based outlier removal
for col in columns_to_clean:
    Q01 = df_clean[col].quantile(0.001)  # 0.1st percentile
    Q99 = df_clean[col].quantile(0.999)  # 99.9th percentile
    
    # Remove rows where the values in the column are outside the 1st and 99th percentile bounds
    df_clean = df_clean[(df_clean[col] >= Q01) & (df_clean[col] <= Q99)]

# Display the cleaned DataFrame
df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000
mean,42.777410,0.655837,0.700642,5.382139,-6.662893,0.565375,0.106136,0.171616,0.081542,0.190224,0.512763,120.739909,225714.729836
std,24.001827,0.137427,0.169031,3.476983,2.737374,0.476230,0.094725,0.205971,0.209185,0.146806,0.223350,25.514146,56360.793917
min,0.000000,0.166000,0.075000,0.000000,-20.338000,0.000000,0.023900,0.000009,0.000000,0.021500,0.035300,62.829000,72000.000000
25%,27.000000,0.575000,0.598000,2.000000,-7.906000,0.000000,0.042100,0.017800,0.000000,0.095400,0.348000,100.085000,190345.000000
50%,43.000000,0.657000,0.701000,5.377047,-6.399000,1.000000,0.068200,0.096100,0.000038,0.138000,0.511568,120.757315,220480.000000
75%,61.000000,0.753000,0.830000,8.000000,-4.761000,1.000000,0.121000,0.228000,0.026800,0.230000,0.680000,130.084000,249063.000000
max,100.000000,0.966000,0.995000,11.000000,-0.902000,1.000000,0.609000,0.969000,0.958000,0.968000,0.974000,205.586000,506200.000000


### Categorical Encoding

In [98]:
# Corrected bins for track_popularity (7 edges, 6 labels)
popularity_bins = [0, 10, 30, 50, 70, 90, 100]
popularity_labels = ['Very Low Popularity', 'Low Popularity', 'Medium-Low Popularity', 'Medium Popularity', 'High Popularity', 'Very High Popularity']
df_clean['popularity_category'] = pd.Categorical(pd.cut(df_clean['track_popularity'], bins=popularity_bins, labels=popularity_labels))

# Corrected bins for tempo (8 edges, 7 labels)
tempo_bins = [60, 80, 100, 120, 140, 160, 180, 205]
tempo_labels = ['Very Slow', 'Slow', 'Moderate', 'Fast', 'Very Fast', 'Extremely Fast', 'Ultra Fast']
df_clean['tempo_category'] = pd.Categorical(pd.cut(df_clean['tempo'], bins=tempo_bins, labels=tempo_labels))


# Corrected bins for loudness (5 edges, 4 labels)
loudness_bins = [-21, -15, -10, -5, 0]
loudness_labels = ['Very Quiet', 'Quiet', 'Moderate', 'Loud']
df_clean['loudness_category'] = pd.Categorical(pd.cut(df_clean['loudness'], bins=loudness_bins, labels=loudness_labels))
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category
1,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,pop,dance pop,0.726000,0.815,...,0.03730,0.072400,0.004210,0.357000,0.693,120.757315,162600.0,Medium Popularity,Fast,Loud
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718000,0.930,...,0.10200,0.028700,0.000009,0.204000,0.277,121.956000,169093.0,Medium Popularity,Fast,Loud
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.0,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650000,0.833,...,0.03590,0.080300,0.000000,0.191022,0.725,123.976000,189052.0,Medium Popularity,Fast,Loud
6,Never Really Over - R3HAB Remix,Katy Perry,62.0,Never Really Over (R3HAB Remix),2019-07-26,Pop Remix,pop,dance pop,0.449000,0.856,...,0.10645,0.187000,0.000000,0.176000,0.152,112.648000,187675.0,Medium Popularity,Moderate,Loud
7,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.0,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29,Pop Remix,pop,dance pop,0.542000,0.903,...,0.04340,0.033500,0.000005,0.191022,0.367,127.936000,207619.0,Medium Popularity,Fast,Loud
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32824,Hey Brother,Avicii,20.0,True,2013-01-01,♥ EDM LOVE 2020,edm,progressive electro house,0.654245,0.780,...,0.04360,0.030900,0.000046,0.082800,0.458,125.014000,255093.0,Low Popularity,Fast,Loud
32827,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,27.0,Many Ways,2013,♥ EDM LOVE 2020,edm,progressive electro house,0.654245,0.640,...,0.03650,0.026600,0.000000,0.572000,0.288,128.001000,196993.0,Low Popularity,Fast,Moderate
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.0,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,edm,progressive electro house,0.428000,0.922,...,0.09360,0.174117,0.000000,0.066800,0.210,128.170000,204375.0,Medium-Low Popularity,Fast,Loud
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20.0,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,edm,progressive electro house,0.522000,0.786,...,0.04200,0.001710,0.004270,0.375000,0.400,128.041000,353120.0,Low Popularity,Fast,Loud


## Feature splitting for release_date (Domain expertise)

In [99]:
# Convert track_album_release_date to datetime, coercing errors
df_clean['track_album_release_date'] = pd.to_datetime(df_clean['track_album_release_date'], errors='coerce')

# Extract the month name and create a new column
df_clean['release_month'] = df_clean['track_album_release_date'].dt.strftime('%B')

# Display the DataFrame
df_clean.drop(columns=['instrumentalness', 'acousticness', 'mode', 'playlist_subgenre', 'playlist_name'], inplace=True)
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
1,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13,pop,0.726000,0.815,11.000000,-4.969000,0.03730,0.357000,0.693,120.757315,162600.0,Medium Popularity,Fast,Loud,December
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,pop,0.718000,0.930,7.000000,-3.778000,0.10200,0.204000,0.277,121.956000,169093.0,Medium Popularity,Fast,Loud,July
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.0,Someone You Loved (Future Humans Remix),2019-03-05,pop,0.650000,0.833,1.000000,-4.672000,0.03590,0.191022,0.725,123.976000,189052.0,Medium Popularity,Fast,Loud,March
6,Never Really Over - R3HAB Remix,Katy Perry,62.0,Never Really Over (R3HAB Remix),2019-07-26,pop,0.449000,0.856,5.000000,-4.788000,0.10645,0.176000,0.152,112.648000,187675.0,Medium Popularity,Moderate,Loud,July
7,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.0,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29,pop,0.542000,0.903,4.000000,-2.419000,0.04340,0.191022,0.367,127.936000,207619.0,Medium Popularity,Fast,Loud,August
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32824,Hey Brother,Avicii,20.0,True,2013-01-01,edm,0.654245,0.780,7.000000,-4.867000,0.04360,0.082800,0.458,125.014000,255093.0,Low Popularity,Fast,Loud,January
32827,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,27.0,Many Ways,NaT,edm,0.654245,0.640,5.000000,-8.367000,0.03650,0.572000,0.288,128.001000,196993.0,Low Popularity,Fast,Moderate,NaN
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.0,City Of Lights (Vocal Mix),2014-04-28,edm,0.428000,0.922,2.000000,-1.814000,0.09360,0.066800,0.210,128.170000,204375.0,Medium-Low Popularity,Fast,Loud,April
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20.0,Closer Remixed,2013-03-08,edm,0.522000,0.786,5.377047,-4.462000,0.04200,0.375000,0.400,128.041000,353120.0,Low Popularity,Fast,Loud,March


In [100]:
df_clean['track_album_release_date'] = df_clean['track_album_release_date'].fillna(0)
df_clean = df_clean[df_clean['track_album_release_date'] != '0']
df_clean.describe()


,track_popularity,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms
count,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000,20913.000000
mean,42.777410,0.655837,0.700642,5.382139,-6.662893,0.106136,0.190224,0.512763,120.739909,225714.729836
std,24.001827,0.137427,0.169031,3.476983,2.737374,0.094725,0.146806,0.223350,25.514146,56360.793917
min,0.000000,0.166000,0.075000,0.000000,-20.338000,0.023900,0.021500,0.035300,62.829000,72000.000000
25%,27.000000,0.575000,0.598000,2.000000,-7.906000,0.042100,0.095400,0.348000,100.085000,190345.000000
50%,43.000000,0.657000,0.701000,5.377047,-6.399000,0.068200,0.138000,0.511568,120.757315,220480.000000
75%,61.000000,0.753000,0.830000,8.000000,-4.761000,0.121000,0.230000,0.680000,130.084000,249063.000000
max,100.000000,0.966000,0.995000,11.000000,-0.902000,0.609000,0.968000,0.974000,205.586000,506200.000000


In [101]:
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
1,Memories - Dillon Francis Remix,Maroon 5,67.0,Memories (Dillon Francis Remix),2019-12-13 00:00:00,pop,0.726000,0.815,11.000000,-4.969000,0.03730,0.357000,0.693,120.757315,162600.0,Medium Popularity,Fast,Loud,December
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19 00:00:00,pop,0.718000,0.930,7.000000,-3.778000,0.10200,0.204000,0.277,121.956000,169093.0,Medium Popularity,Fast,Loud,July
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69.0,Someone You Loved (Future Humans Remix),2019-03-05 00:00:00,pop,0.650000,0.833,1.000000,-4.672000,0.03590,0.191022,0.725,123.976000,189052.0,Medium Popularity,Fast,Loud,March
6,Never Really Over - R3HAB Remix,Katy Perry,62.0,Never Really Over (R3HAB Remix),2019-07-26 00:00:00,pop,0.449000,0.856,5.000000,-4.788000,0.10645,0.176000,0.152,112.648000,187675.0,Medium Popularity,Moderate,Loud,July
7,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.0,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29 00:00:00,pop,0.542000,0.903,4.000000,-2.419000,0.04340,0.191022,0.367,127.936000,207619.0,Medium Popularity,Fast,Loud,August
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32824,Hey Brother,Avicii,20.0,True,2013-01-01 00:00:00,edm,0.654245,0.780,7.000000,-4.867000,0.04360,0.082800,0.458,125.014000,255093.0,Low Popularity,Fast,Loud,January
32827,Many Ways - Radio Edit,Ferry Corsten feat. Jenny Wahlstrom,27.0,Many Ways,0,edm,0.654245,0.640,5.000000,-8.367000,0.03650,0.572000,0.288,128.001000,196993.0,Low Popularity,Fast,Moderate,NaN
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.0,City Of Lights (Vocal Mix),2014-04-28 00:00:00,edm,0.428000,0.922,2.000000,-1.814000,0.09360,0.066800,0.210,128.170000,204375.0,Medium-Low Popularity,Fast,Loud,April
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20.0,Closer Remixed,2013-03-08 00:00:00,edm,0.522000,0.786,5.377047,-4.462000,0.04200,0.375000,0.400,128.041000,353120.0,Low Popularity,Fast,Loud,March


In [102]:
df_clean.to_csv("./dataset/spotify_songs_restructured_cleaned.csv", index=False)